# OpenAI API 강의 6일차 — RAG + LangChain 실습 완전판


## 학습 목표
- RAG(Retrieval-Augmented Generation)의 개념과 구조 이해
- OpenAI Embeddings API로 벡터화 및 코사인 유사도 계산 실습
- LangChain 설치 및 핵심 컴포넌트(LLM, PromptTemplate, Chain) 이해
- FAISS/Chroma 기반 벡터 검색 + RetrievalQA, ConversationalRetrievalChain 구축
- PDF 문서 기반 RAG 질의응답 시스템 구현



## 1. RAG 개념 및 구조
| 항목 | 설명 |
|---|---|
| 정의 | 외부 지식원(DB/문서/웹)을 검색(Retrieval)해 LLM 생성(Generation)에 반영하는 아키텍처 |
| 핵심 구성 | Retriever(검색기) + Generator(생성기) |
| 장점 | 최신 정보 반영, 환각 감소, 도메인 특화 정확도 향상 |
| 활용 | 사내 위키 Q&A, 매뉴얼/정책 문서 질의응답, 제품 FAQ 챗봇 |

기본 흐름:
```
사용자 질문 → [Retriever] 관련 문서 검색 → [Generator] 문서 기반 답변 생성
```



## 2. OpenAI Embeddings 실습: 벡터화와 코사인 유사도
아래 예제는 텍스트를 임베딩으로 변환하고, 코사인 유사도로 유사도를 계산합니다.


In [1]:

from openai import OpenAI
import numpy as np

client = OpenAI()

texts = [
    "OpenAI는 인공지능 연구소입니다.",
    "RAG는 검색 기반 생성 모델 구조를 의미합니다.",
    "벡터 데이터베이스는 문장 임베딩을 저장합니다."
]

# 문장 임베딩 생성
embs = []
for t in texts:
    context_vector = client.embeddings.create(input=t, model="text-embedding-3-small").data[0].embedding
    embs.append(context_vector)
    print(len(context_vector))
    print(context_vector)
# embs = [client.embeddings.create(input=t, model="text-embedding-3-small").data[0].embedding for t in texts]

1536
[-0.007375389337539673, -0.004468097351491451, 0.019307399168610573, 0.013297133147716522, 0.021469231694936752, -0.015906240791082382, -0.056915830820798874, 0.03563296049833298, 0.014583050273358822, -0.03803706914186478, -0.031067023053765297, -0.027656545862555504, 0.010837117210030556, -0.04368392378091812, -0.011442802846431732, -0.018971942365169525, 0.00273490441031754, 0.0015479931607842445, 0.0006505298661068082, -0.02689244970679283, -0.028178367763757706, -0.007748119067400694, -0.014135775156319141, -0.012393263168632984, 0.015151463449001312, -0.0552758164703846, 0.052964892238378525, 0.012393263168632984, -0.009746882133185863, -0.0040417881682515144, 0.04838031902909279, -0.021506505087018013, -0.04454120248556137, -0.014107819646596909, 0.0041932095773518085, 0.041596636176109314, 0.02087286487221718, 0.007291525136679411, -0.005045828875154257, -0.005931061692535877, 0.004195538815110922, -0.008861648850142956, -0.012095079757273197, 0.034048859030008316, -0.0189

In [2]:
def cosine_similarity(a, b):
    a = np.array(a); b = np.array(b)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

# 질의 임베딩
query = "RAG 구조에 대해 설명해줘."
q_emb = client.embeddings.create(input=query, model="text-embedding-3-small").data[0].embedding

# 유사도 계산 및 상위 정렬
scores = [cosine_similarity(q_emb, e) for e in embs]
for idx, score in sorted(enumerate(scores), key=lambda x: x[1], reverse=True):
    print(f"{score:.3f} : {texts[idx]}")


0.722 : RAG는 검색 기반 생성 모델 구조를 의미합니다.
0.249 : 벡터 데이터베이스는 문장 임베딩을 저장합니다.
0.220 : OpenAI는 인공지능 연구소입니다.



## 3. LangChain 설치
아래 명령으로 필요한 라이브러리를 설치하세요.


In [3]:

# 터미널이나 노트북에서 실행
# %pip install langchain openai faiss-cpu chromadb pypdf tiktoken
print("필요 패키지: langchain, openai, faiss-cpu, chromadb, pypdf, tiktoken")


필요 패키지: langchain, openai, faiss-cpu, chromadb, pypdf, tiktoken


In [ ]:
# !pip install langchain openai faiss-cpu chromadb pypdf tiktoken

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/18.8 MB ? eta -:--:--
   --- ------------------------------------ 1.8/18.8 MB 8.4 MB/s eta 0:00:03
   ------- -------------------------------- 3.7/18.8 MB 9.1 MB/s eta 0:00:02
   ------------ --------------------------- 6.0/18.8 MB 9.5 MB/s eta 0:00:02
   ---------------- ----------------------- 7.6/18.8 MB 9.0 MB/s eta 0:00:02
   --------------------- ------------------ 10.2/18.8 MB 9.4 MB/s eta 0:00:01
   -------------------------- ------------- 12.6/18.8 MB 9.7 MB/s eta 0:00:01
   ------------------------------- -------- 14.9/18.8 MB 9.9 MB/s eta 0:00

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.39.0 requires urllib3[socks]<3.0,>=2.5.0, but you have urllib3 2.3.0 which is incompatible.


  Using cached langchain-1.2.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached chromadb-1.3.7-cp39-abi3-win_amd64.whl.metadata (7.3 kB)
  Using cached langchain_core-1.2.5-py3-none-any.whl.metadata (3.7 kB)
  Using cached langgraph-1.0.5-py3-none-any.whl.metadata (7.4 kB)
  Using cached langgraph_checkpoint-3.0.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached langgraph_prebuilt-1.0.5-py3-none-any.whl.metadata (5.2 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.39.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_sdk-1.39.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached langchain-1.2.0-py3-none-any.whl (102 kB)
Using cached langchain_core-1.2.5-py3-none-any.whl (484 kB)
Using cached langgraph-1.0.5-py3-none-any.whl (157 kB)
Using cached langgraph_checkpoint-3.0.1-py3-none-any.whl (46 kB)
Using cached langgraph_prebuilt-1.0.5-py3-none-any.whl (35 kB)
Using cached chromadb-1.3.7-c


## 4. LangChain 기본 구조: LLM + PromptTemplate + Chain
LLM, PromptTemplate, Chain을 사용해 간단한 파이프라인을 구성합니다.


In [11]:
import langchain
langchain.__version__  # 확인

'1.2.0'

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model="gpt-4o-mini")

template = "너는 친절한 AI 강사야. 다음 질문에 자세히 답해줘: {question}"
prompt = PromptTemplate(template=template, input_variables=["question"])

chain = LLMChain(llm=llm, prompt=prompt)
reply = chain.run("LangChain의 장점은 무엇인가요?")
print(reply)

ImportError: cannot import name 'ChatOpenAI' from 'langchain.chat_models' (d:\Users\admin\anaconda3\envs\py310\lib\site-packages\langchain\chat_models\__init__.py)

In [11]:
from IPython.display import Markdown
Markdown(reply)

LangChain은 자연어 처리(NLP)와 관련된 다양한 기능을 제공하는 프레임워크로, 특히 대화형 AI 및 언어 모델을 활용한 애플리케이션 개발에 유용합니다. LangChain의 장점은 다음과 같습니다:

1. **모듈화**: LangChain은 다양한 구성 요소(모듈)로 나뉘어 있어, 개발자가 필요한 기능만 선택하여 사용할 수 있습니다. 이를 통해 복잡한 시스템을 효율적으로 구축할 수 있습니다.

2. **다양한 언어 모델 지원**: LangChain은 여러 종류의 언어 모델(OpenAI, Hugging Face 등)을 지원합니다. 개발자는 특정 애플리케이션에 가장 적합한 모델을 선택하여 사용할 수 있습니다.

3. **체인 구성**: LangChain은 여러 단계를 체인 형태로 연결하여 복잡한 작업을 수행할 수 있게 해줍니다. 예를 들어, 사용자의 입력을 받아서 처리하고, 그 결과를 기반으로 다른 API에 요청을 하거나, 데이터베이스에서 정보를 조회하는 등의 작업을 순차적으로 수행할 수 있습니다.

4. **상태 관리**: 대화형 애플리케이션에서 사용자와의 상호작용을 보다 매끄럽게 만들기 위해, LangChain은 상태를 관리하는 기능을 제공합니다. 이를 통해 사용자 경험을 개선할 수 있습니다.

5. **확장성**: LangChain은 사용자가 필요에 따라 기능을 쉽게 확장할 수 있는 구조를 가지고 있습니다. 새로운 모델이나 기능을 추가하거나, 기존 구성 요소를 수정하여 맞춤형 솔루션을 만들 수 있습니다.

6. **유연한 데이터 소스 통합**: LangChain은 데이터베이스, API, 파일 시스템 등 다양한 데이터 소스로부터 정보를 손쉽게 통합하고 활용할 수 있는 기능을 제공합니다. 이를 통해 정보의 출처를 다양화할 수 있습니다.

7. **커뮤니티와 문서화**: LangChain은 활발한 커뮤니티와 함께 잘 정리된 문서화가 제공되어 있어, 개발자들이 쉽게 배우고 사용할 수 있습니다. 문제 해결이나 새로운 아이디어를 찾는 데 도움이 됩니다.

8. **고급 기능**: LangChain은 자연어 처리에 필요한 다양한 고급 기능을 제공합니다. 예를 들어, 문서 검색, 요약, 질문 응답 시스템 등을 구축하는 데 필요한 도구들을 제공합니다.

이러한 장점들 덕분에 LangChain은 대화형 AI 및 NLP 기반 애플리케이션 개발에 있어 매우 유용한 도구로 자리매김하고 있습니다.


## 5. 두 개의 Chain 연결: SimpleSequentialChain
한 단계의 결과를 다음 단계 입력으로 연결합니다.


In [ ]:

from langchain.chains import SimpleSequentialChain

# 1단계: 요약
prompt_summary = PromptTemplate(
    template="다음 텍스트를 한 문장으로 요약해줘:\n{content}",
    input_variables=["content"]
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary) # llm = ChatOpenAI(model="gpt-4o-mini")

# 2단계: 영어 번역
prompt_translate = PromptTemplate(
    template="다음 문장을 영어로 번역해줘:\n{content}",
    input_variables=["content"]
)
chain_translate = LLMChain(llm=llm, prompt=prompt_translate)

# 두 체인을 순차적으로 연결
seq_chain = SimpleSequentialChain(chains=[chain_summary, chain_translate])

# 예문
text = """
LangChain은 LLM을 활용한 애플리케이션 개발을 위한 강력한 오픈소스 프레임워크입니다.
이 라이브러리는 개발자가 프롬프트 관리, 체인 구성, 에이전트 생성 등 복잡한 작업을
모듈화된 방식으로 쉽게 처리할 수 있도록 돕습니다. 특히, 외부 데이터 소스를 LLM과
연동하는 RAG(검색 증강 생성) 시스템을 구축할 때 그 진가를 발휘하며,
이는 AI 챗봇의 답변 정확도를 크게 향상시킬 수 있습니다.
"""

# 체인 실행 및 결과 출력
print(seq_chain.run(text))

LangChain is an open-source framework that supports the development of applications using LLMs, modularizing complex tasks for processing, and particularly enhances the accuracy of AI chatbot responses through RAG systems.


In [ ]:
# 요약만 실시
chain_summary.run(text)

'LangChain은 LLM을 활용한 애플리케이션 개발을 지원하는 오픈소스 프레임워크로, 프롬프트 관리, 체인 구성, 에이전트 생성 등을 쉽게 모듈화하여 외부 데이터와 연동하는 RAG 시스템을 통해 AI 챗봇의 답변 정확도를 향상시킵니다.'

In [ ]:
# 번역만 실시
chain_translate.run(text)

'LangChain is a powerful open-source framework for developing applications that utilize LLMs. This library helps developers handle complex tasks such as prompt management, chain construction, and agent creation in a modular way. In particular, it shines when building RAG (Retrieval-Augmented Generation) systems that integrate external data sources with LLMs, significantly enhancing the accuracy of responses from AI chatbots.'


## 6. Embedding + FAISS 벡터DB로 유사 검색
텍스트를 벡터화하여 FAISS에 저장하고, 질의로 유사한 문서를 검색합니다.


#### ※ 점수(Score) 해석 방법

`similarity_search_with_score`가 반환하는 점수는 일반적인 '유사도'와는 조금 다릅니다. 이 점수는 두 벡터 사이의 **거리(distance)** 를 의미합니다.

FAISS의 기본 거리 측정 방식은 **L2 거리(유클리드 거리)** 입니다.

  * **점수가 0에 가까울수록 (즉, 작을수록) 더 유사하다**는 의미입니다.
  * 반대로 점수가 클수록 두 벡터의 거리가 멀어 유사성이 낮다는 뜻입니다.

따라서 위 코드를 실행하면, 검색어와 가장 관련성이 높은 문서가 가장 작은 점수와 함께 출력될 것입니다.

In [ ]:
# !pip install langchain_openai

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# OpenAIEmbeddings 객체 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# --- 확장된 문서 리스트 ---
docs = [
    "RAG는 검색과 생성을 결합하여 LLM의 답변 정확도를 높이는 모델 구조입니다.",
    "FAISS는 Meta에서 개발한, 대규모 벡터를 빠르게 검색하기 위한 라이브러리입니다.",
    "Chroma는 파이썬에서 쉽게 쓸 수 있는 경량 오픈소스 벡터 데이터베이스입니다.",
    "임베딩은 텍스트나 이미지 같은 데이터를 컴퓨터가 이해할 수 있는 숫자 벡터로 변환하는 과정입니다.",
    "벡터 저장소는 임베딩된 벡터들을 저장하고, 빠르고 효율적인 유사도 검색을 가능하게 합니다.",
    "LangChain은 LLM을 활용한 애플리케이션 개발을 돕는 강력한 프레임워크입니다.",
    "에이전트는 LLM이 스스로 판단하여 외부 도구를 사용하는 등 복잡한 작업을 수행하는 능력을 가집니다.",
    "파인튜닝은 사전 학습된 거대 언어 모델을 특정 작업이나 도메인에 맞게 추가로 학습시키는 과정입니다.",
    "시맨틱 검색은 단순한 키워드 일치가 아닌, 문장의 문맥과 의미를 기반으로 검색 결과를 제공합니다.",
    "허깅페이스는 트랜스포머 기반의 다양한 모델과 데이터셋을 공유하는 거대한 플랫폼입니다.",
    "Pinecone은 클라우드 기반의 완전 관리형 벡터 데이터베이스 서비스로, 확장성이 뛰어납니다.",
    "LlamaIndex는 LLM에 외부 데이터를 연결하고 질의하는 것에 특화된 데이터 프레임워크입니다."
]

# Faiss 벡터DB 생성
print("벡터 데이터베이스를 생성 중입니다...")
db = FAISS.from_texts(docs, embeddings)
print("생성 완료!")


# --- 다양한 검색어로 테스트 ---
queries = [
    "벡터 데이터베이스 종류 알려줘",
    "LLM한테 외부 지식을 알려주려면 어떻게 해?",
    "모델을 특정 목적에 맞게 훈련시키는 건 뭐야?"
]

for query in queries:
    print("\n" + "="*40)
    print(f"검색어: '{query}'")
    print("="*40)
    
    # 유사도 검색 실행 (점수와 함께)
    results_with_scores = db.similarity_search_with_score(query, k=3)
    
    # 결과 출력
    for doc, score in results_with_scores:
        print(f"Score: {score:.4f}") # 소수점 4자리까지 출력
        print(f"Content: {doc.page_content}")
        print("-" * 20)

벡터 데이터베이스를 생성 중입니다...
생성 완료!

검색어: '벡터 데이터베이스 종류 알려줘'
Score: 0.8647
Content: 벡터 저장소는 임베딩된 벡터들을 저장하고, 빠르고 효율적인 유사도 검색을 가능하게 합니다.
--------------------
Score: 1.1577
Content: Pinecone은 클라우드 기반의 완전 관리형 벡터 데이터베이스 서비스로, 확장성이 뛰어납니다.
--------------------
Score: 1.1616
Content: Chroma는 파이썬에서 쉽게 쓸 수 있는 경량 오픈소스 벡터 데이터베이스입니다.
--------------------

검색어: 'LLM한테 외부 지식을 알려주려면 어떻게 해?'
Score: 1.1289
Content: 에이전트는 LLM이 스스로 판단하여 외부 도구를 사용하는 등 복잡한 작업을 수행하는 능력을 가집니다.
--------------------
Score: 1.1923
Content: LlamaIndex는 LLM에 외부 데이터를 연결하고 질의하는 것에 특화된 데이터 프레임워크입니다.
--------------------
Score: 1.2822
Content: RAG는 검색과 생성을 결합하여 LLM의 답변 정확도를 높이는 모델 구조입니다.
--------------------

검색어: '모델을 특정 목적에 맞게 훈련시키는 건 뭐야?'
Score: 1.3431
Content: 파인튜닝은 사전 학습된 거대 언어 모델을 특정 작업이나 도메인에 맞게 추가로 학습시키는 과정입니다.
--------------------
Score: 1.4721
Content: 에이전트는 LLM이 스스로 판단하여 외부 도구를 사용하는 등 복잡한 작업을 수행하는 능력을 가집니다.
--------------------
Score: 1.4900
Content: 임베딩은 텍스트나 이미지 같은 데이터를 컴퓨터가 이해할 수 있는 숫자 벡터로 변환하는 과정입니다.
--------------------



## 7. RetrievalQA로 RAG 파이프라인 구성
벡터DB에서 검색한 문맥으로 LLM이 답하도록 구성합니다.


* RetrievalQA는 LLM이 엉뚱한 대답(환각)을 하는 대신, 주어진 자료에 근거하여 정확하고 신뢰도 높은 답변을 하도록 유도합니다.

### 동작 과정
RetrievalQA 체인은 내부적으로 다음과 같은 단계를 거쳐 작동합니다.

**1. 질문 입력**: 사용자가 질문을 합니다. (예: "FAISS의 장점은 무엇인가요?")

**2. 문서 검색 (Retrieve)**:

* 질문 문장을 벡터로 변환(임베딩)합니다.

* 미리 생성해 둔 벡터 저장소(Vector Store, 예: FAISS)에서 질문 벡터와 가장 유사한(관련성 높은) 문서 조각들을 검색합니다.

**3. 프롬프트 구성 (Augment):**

* 검색된 문서 조각들과 원래 질문을 조합하여 LLM에게 전달할 새로운 프롬프트(Prompt)를 만듭니다.

* 예시:
```yaml
    [Context]
    - FAISS는 Meta에서 개발한... 라이브러리입니다.
    - FAISS는 메모리 효율성이 뛰어납니다...

    [Question]
    FAISS의 장점은 무엇인가요?
```
**4. 답변 생성 (Generate):**

구성된 프롬프트를 LLM(예: GPT-4)에 전달합니다.

LLM은 주어진 Context 내용을 바탕으로 질문에 대한 최종 답변을 생성합니다.

In [23]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

question = "RAG와 FAISS의 역할을 설명해줘."
res = qa.invoke(question)
res

{'query': 'RAG와 FAISS의 역할을 설명해줘.',
 'result': 'RAG(검색과 생성 결합 모델)는 LLM의 답변 정확도를 높이는 모델 구조로, 정보 검색과 생성 과정을 통합하여 보다 정확한 답변을 제공합니다. \n\nFAISS는 Meta에서 개발한 라이브러리로, 대규모 벡터를 빠르게 검색할 수 있도록 도와줍니다. 주로 RAG와 같은 모델에서 대량의 데이터를 효율적으로 검색하는 데 사용됩니다. \n\n즉, RAG는 데이터를 검색하고 생성하는 구조를 제공하며, FAISS는 이 과정에서 필요한 데이터를 빠르게 찾는 역할을 합니다.',
 'source_documents': [Document(id='aa694835-7aa1-4930-a3de-97d5b2ac177b', metadata={}, page_content='FAISS는 Meta에서 개발한, 대규모 벡터를 빠르게 검색하기 위한 라이브러리입니다.'),
  Document(id='075fb964-46a7-4ebe-b33f-aa8134f7592c', metadata={}, page_content='RAG는 검색과 생성을 결합하여 LLM의 답변 정확도를 높이는 모델 구조입니다.'),
  Document(id='9d9ce479-652a-4c3a-90c4-54dd1ae69c6b', metadata={}, page_content='LlamaIndex는 LLM에 외부 데이터를 연결하고 질의하는 것에 특화된 데이터 프레임워크입니다.'),
  Document(id='b7aa97fb-f380-46e0-9ccf-1e225a91e51d', metadata={}, page_content='에이전트는 LLM이 스스로 판단하여 외부 도구를 사용하는 등 복잡한 작업을 수행하는 능력을 가집니다.')]}

In [ ]:
print("답변:", res["result"])

답변: RAG(검색과 생성 결합 모델)는 LLM의 답변 정확도를 높이는 모델 구조로, 정보 검색과 생성 과정을 통합하여 보다 정확한 답변을 제공합니다. 

FAISS는 Meta에서 개발한 라이브러리로, 대규모 벡터를 빠르게 검색할 수 있도록 도와줍니다. 주로 RAG와 같은 모델에서 대량의 데이터를 효율적으로 검색하는 데 사용됩니다. 

즉, RAG는 데이터를 검색하고 생성하는 구조를 제공하며, FAISS는 이 과정에서 필요한 데이터를 빠르게 찾는 역할을 합니다.


In [25]:
print("\n참고 소스 문서:")



참고 소스 문서:


In [26]:
for d in res["source_documents"]:
    print("-", d.page_content)


- FAISS는 Meta에서 개발한, 대규모 벡터를 빠르게 검색하기 위한 라이브러리입니다.
- RAG는 검색과 생성을 결합하여 LLM의 답변 정확도를 높이는 모델 구조입니다.
- LlamaIndex는 LLM에 외부 데이터를 연결하고 질의하는 것에 특화된 데이터 프레임워크입니다.
- 에이전트는 LLM이 스스로 판단하여 외부 도구를 사용하는 등 복잡한 작업을 수행하는 능력을 가집니다.



## 8. ConversationalRetrievalChain로 대화형 RAG
대화 기록을 메모리로 관리하면서 검색+생성을 수행합니다.


In [27]:

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conv = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(),
    memory=memory
)

qs = ["RAG는 무엇인가요?", "FAISS는 왜 쓰나요?", "둘의 관계를 간단히 요약해줘."]
for q in qs:
    out = conv.invoke({"question": q})
    print("Q:", q)
    print("A:", out["answer"], "\n")


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_37628\3769732185.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


Q: RAG는 무엇인가요?
A: RAG는 검색과 생성을 결합하여 LLM(대형 언어 모델)의 답변 정확도를 높이는 모델 구조입니다. 

Q: FAISS는 왜 쓰나요?
A: FAISS는 대규모 벡터를 빠르게 검색하기 위한 라이브러리로 사용됩니다. 주로 임베딩된 벡터들을 저장하고, 이들 간의 유사도를 효율적으로 검색하는 데 최적화되어 있습니다. 따라서 대량의 데이터에서 유사한 항목을 빠르게 찾고자 할 때 유용하게 사용됩니다. 

Q: 둘의 관계를 간단히 요약해줘.
A: RAG는 검색과 생성을 결합하여 LLM의 답변 정확도를 높이는 모델 구조이고, FAISS는 대규모 벡터를 빠르게 검색하기 위한 라이브러리입니다. RAG 모델은 FAISS와 같은 검색 라이브러리를 활용하여 필요한 정보를 빠르게 검색하고, 이를 바탕으로 LLM이 보다 정확한 답변을 생성할 수 있도록 돕습니다. 따라서 RAG는 FAISS를 사용할 수 있는 구조로, 서로 보완적인 관계에 있습니다. 




## 9. PDF 문서 기반 RAG 질의응답
PyPDFLoader로 PDF를 로드하고, 벡터DB를 구성해 질의응답을 수행합니다.


In [28]:

from langchain.document_loaders import PyPDFLoader

# 예시 파일명: example.pdf (같은 폴더에 준비)
loader = PyPDFLoader("카카오뱅크323410.pdf")
pages = loader.load()

texts = [p.page_content for p in pages]
pdf_db = FAISS.from_texts(texts, embeddings)
pdf_retriever = pdf_db.as_retriever()

qa_pdf = RetrievalQA.from_chain_type(llm=llm, retriever=pdf_retriever)
print(qa_pdf.invoke("이 문서의 핵심 내용을 요약해줘.")["result"])


이 문서는 카카오뱅크의 재무 성과와 투자 전망에 대한 분석을 담고 있습니다. 2023년 카카오뱅크는 당기순이익이 3,549억원으로 전년 대비 34.9% 증가하였으며, 이자수익도 증가하였습니다. 4분기 순이익은 전분기 대비 감소했지만 연간 실적은 양호하다고 평가됩니다. 2024년에도 대출 성장이 예상되며, 플랫폼 수익 증가도 긍정적인 요소로 작용할 것으로 보입니다. 투자의견은 '매수'로 유지되며 목표 주가는 37,000원으로 설정되었습니다.



## 10. 실습 과제
1) 자신의 문서를 TXT/PDF로 준비해 벡터DB 구축 후 질의응답을 구성하세요.  
2) Retrieval 파라미터(k, score_threshold 등)를 조정해 정확도 변화를 비교하세요.  
3) 프롬프트에 인용 표기 형식(예: [출처: 문서제목])을 강제하여 근거를 포함한 답변을 생성해보세요.  

## 참고 자료
- OpenAI Embeddings API: https://platform.openai.com/docs/api-reference/embeddings
- LangChain 공식 문서: https://python.langchain.com/docs/
- FAISS: https://faiss.ai/
- LangChain PDF QA 튜토리얼: https://python.langchain.com/docs/tutorials/pdf_qa
